# Introducing Qiskit Nature to study energy level
## Implementation part

### I. The Hartree-Fock initial state

As we obtained from the theoretical derivation of the literature part, we have the Molecular Orbitals (MOs) or the Hamiltonian re-expressed in the basis of the solution of the Hartree-Fock (HF) method as:

$$\hat{H}_{elec}=\sum_{pq} h_{pq} \hat{a}^{\dagger}_p \hat{a}_q + \frac{1}{2} \sum_{pqrs} h_{pqrs}  \hat{a}^{\dagger}_p \hat{a}^{\dagger}_q \hat{a}_r  \hat{a}_s$$

The platform we use in this project, Qiskit Nature, is interfaced with different classical codes, which are able to find the HF solutions. Interfacing between Qiskit Nature and the following codes is already available: "Gaussian", "Psi4", "PyQuante" and "PySCF". In the following I will set up a PySCF driver for the hydrogen molecule at equilibrium bond length (0.735 angstrom) in the singlet state and with no charge.

In [1]:
#We have to first install the linting tools in Jupyter notebook
!pip install flake8 pycodestyle_magic
#To enable automatic linting use cell magic:
%load_ext pycodestyle_magic
%pycodestyle_on

In [2]:
from qiskit_nature.drivers import UnitsType, Molecule
from qiskit_nature.drivers.second_quantization import ElectronicStructureDriverType, ElectronicStructureMoleculeDriver
molecule = Molecule(geometry=[['H', [0., 0., 0.]],
                              ['H', [0., 0., 0.735]]], charge=0, multiplicity=1)
"""This is a docstring for the imported function Molecule
   Information below is based on the docstring in molecule.py

   Description
   -----------

   This module implements an interface for a driver-independent, i.e. generic
   molecule definition. It defines the composing atoms (with properties like
   masses), and allows for changing the molecular geometry through given
   degrees of freedom (e.g. bond-stretching, angle-bending, etc.). The
   geometry as provided in the constructor can be affected, through setting
   perturbations, and it is this perturbed geometry that is supplied by the
   geometry getter. Setting perturbations to None will cause the original
   geometry to be returned, and there is a getter to get this value directly
   if its needed.
"""

"""This is a docstring for the imported
   function ElectronicStructureMoleculeDriver
   Information below is based on the docstring
   in electronic_structure_molecule_driver.py

   Description
   -----------

   Molecule based electronic structure driver
"""
driver = ElectronicStructureMoleculeDriver(molecule, basis='sto3g', driver_type=ElectronicStructureDriverType.PYSCF)

/opt/conda/lib/python3.8/site-packages/pyscf/lib/misc.py:47: H5pyDeprecationWarning: Using default_file_mode other than 'r' is deprecated. Pass the mode to h5py.File() instead.
  h5py.get_config().default_file_mode = 'a'
2:80: E501 line too long (118 > 79 characters)
4:80: E501 line too long (80 > 79 characters)
32:80: E501 line too long (116 > 79 characters)


### II. The mapping from fermions to qubits

As we mentioned in the literature part, the Hamiltonian we have for HF method is expressed in terms of fermionic operators, and here we will map these operators to spin operators so that we can encode the problem into the state of a quantum computer. Here we set up the Electronic Structure Problem to generate the Second quantized operator and a qubit converter that will map it to a qubit operator.

In [3]:
from qiskit_nature.problems.second_quantization import ElectronicStructureProblem
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.mappers.second_quantization import JordanWignerMapper, ParityMapper

1:80: E501 line too long (81 > 79 characters)
3:80: E501 line too long (86 > 79 characters)


In [4]:
es_problem = ElectronicStructureProblem(driver)
"""This is a docstring for the imported function ElectronicStructureProblem
   Information below is based on the
   docstring in electronic_structure_problem.py

   Description
   -----------

   Electronic Structure Problem
"""
second_q_op = es_problem.second_q_ops()
"""This is a docstring for the function list
   Information below is based on the original docstring

   Description
   -----------

   Built-in mutable sequence.
   If no argument is given, the constructor creates a new empty list.
   The argument must be an iterable if specified.
"""

"""This is a docstring for the function print
   Information below is based on the original docstring

   Description
   -----------

   Prints the values to a stream, or to sys.stdout by default.
"""
print(second_q_op[0])

Fermionic Operator
register length=4, number terms=14
  (0.18093119978423147+0j) * ( +_0 -_1 +_2 -_3 )
+ (-0.18093119978423144+0j) * ( +_0 -_1 -_2 +_3 )
+ (-0.18093119978423144+0j) * ( -_0 +_1 +_2 -_3 )
+ (0.1809311997842317+0j) * ( -_0 +_1 -_2 +_3 ) ...


If we now transform this Hamiltonian for the given driver defined above we get our qubit operator:

In [5]:
qubit_converter = QubitConverter(mapper=JordanWignerMapper())
"""This is a docstring for the imported function QubitConverter
   Information below is based on the docstring in qubit_converter.py

   Description
   -----------

   A converter from Second-Quantized to Qubit Operators. This converter can be
   configured with a mapper instance which will later be used when 2nd
   quantized operators are requested to be converted (mapped) to qubit
   operators.
"""
qubit_op = qubit_converter.convert(second_q_op[0])
"""This is a docstring for the function PauliSumOp
   Information below is based on the docstring in pauli_sum_op.py

   Description
   -----------

   Class for Operators backend by Terra's ``SparsePauliOp`` class.
"""
print(qubit_op)

-0.8105479805373264 * IIII
- 0.22575349222402508 * ZIII
+ 0.17218393261915543 * IZII
+ 0.1209126326177664 * ZZII
- 0.22575349222402505 * IIZI
+ 0.17464343068300475 * ZIZI
+ 0.16614543256382425 * IZZI
+ 0.17218393261915552 * IIIZ
+ 0.16614543256382425 * ZIIZ
+ 0.16892753870087926 * IZIZ
+ 0.1209126326177664 * IIZZ
+ 0.045232799946057875 * XXXX
+ 0.045232799946057875 * YYXX
+ 0.045232799946057875 * XXYY
+ 0.045232799946057875 * YYYY


Note that in the minimal (STO-3G) basis set 4 qubits are required. We can reduce the number of qubits by using the Parity mapping, which allows for the removal of 2 qubits by exploiting known symmetries arising from the mapping.

In [6]:
qubit_converter = QubitConverter(mapper=ParityMapper(), two_qubit_reduction=True)
# The docstring for the function QubitConverter is the same as above

qubit_op = qubit_converter.convert(second_q_op[0], num_particles=es_problem.num_particles)
"""This is a docstring for the function TaperedPauliSumOp
   Information below is based on the docstring in tapered_pauli_sum_op.py

   Description
   -----------

   Class for PauliSumOp after tapering
"""
print(qubit_op)

1:80: E501 line too long (81 > 79 characters)
4:80: E501 line too long (90 > 79 characters)


-1.0523732457728587 * II
+ (-0.3979374248431804+1.3877787807814457e-17j) * ZI
+ 0.3979374248431804 * IZ
+ (-0.01128010425623549+1.3877787807814457e-17j) * ZZ
+ 0.1809311997842315 * XX


As we can see, this time only 2 qubits are needed. Now that the Hamiltonian is ready, it can be used in a quantum algorithm to find information about the electronic structure of the corresponding molecule.

### III. The ground state solver

Since now we have the desired Hamiltonian of the electronic structure, we can discuss the ground state calculation interface and our next goal is to compute the ground state of a molecular Hamiltonian. Since we alreday defined the molecular system, we then need to define a solver, which is the algorithm through which the ground state is computed. Let’s first start with a purely classical example: the NumPy minimum eigensolver. This algorithm exactly diagonalizes the Hamiltonian. Although it scales badly, it can be used on small systems to check the results of the quantum algorithms.

In [7]:
from qiskit.algorithms import NumPyMinimumEigensolver
"""This is a docstring for the imported function NumPyMinimumEigensolver
   Information below is based on the docstring in numpy_minimum_eigen_solver.py

   Description
   -----------

   The Numpy Minimum Eigensolver algorithm.The minimum eigensolver is only
   searching over feasible states and returns an eigenstate that has the
   smallest eigenvalue among feasible states.
"""
numpy_solver = NumPyMinimumEigensolver()

To find the ground state we could also use the Variational Quantum Eigensolver (VQE) algorithm, which works by exchanging information between a classical and a quantum computer as depicted in the following figure.

<img src="vqe.png" alt="comparision fig.2" height="70%" width="70%"/>

To define the VQE solver, these essential elements are required:
1. A variational form: here we use the Unitary Coupled Cluster (UCC) ansatz, which is a chemistry standard, thus a factory is available allowing a fast initialization of a VQE with UCC. The default is to use all single and double excitations, but the excitation type (S, D, SD) as well as other parameters can be selected by users.
2. An initial state: the initial state of the qubits. In the factory used above, the qubits are initialized in the Hartree-Fock initial state (the qubits corresponding to occupied MOs are $|1\rangle$ and those corresponding to virtual MOs are $|0\rangle$).
3. The backend: this is the quantum machine on which the right part of the figure above will be performed. Here we ask for the perfect quantum emulator (aer_simulator_statevector).

Then Let’s initialize a VQE solver like following:

In [8]:
from qiskit.providers.aer import StatevectorSimulator
from qiskit import Aer
from qiskit.utils import QuantumInstance
from qiskit_nature.algorithms import VQEUCCFactory

quantum_instance = QuantumInstance(backend=Aer.get_backend('aer_simulator_statevector'))
"""This is a docstring for the imported function QuantumInstance
   Information below is based on the docstring in quantum_instance.py

   Description
   -----------

   Quantum Backend including execution setting. Quantum Instance holds a
   Qiskit Terra backend as well as configuration for circuit transpilation
   and execution. When provided to an Aqua algorithm the algorithm will
   execute the circuits it needs to run using the instance.
"""

"""This is a docstring for the imported function VQEUCCFactory
   Information below is based on the docstring in vqe_ucc_factory.py

   Description
   -----------

   A factory to construct a VQE minimum eigensolver with UCCSD
   ansatz wavefunction.
"""
vqe_solver = VQEUCCFactory(quantum_instance)

6:80: E501 line too long (88 > 79 characters)


One could also use any available ansatz / initial state or even define one’s own. For instance:

In [9]:
from qiskit.algorithms import VQE
from qiskit.circuit.library import TwoLocal
tl_circuit = TwoLocal(rotation_blocks=['h', 'rx'], entanglement_blocks='cz',
                      entanglement='full', reps=2, parameter_prefix='y')
"""This is a docstring for the imported function TwoLocal
   Information below is based on the docstring in two_local.py

   Description
   -----------

   The two-local circuit, which is a parameterized circuit consisting of
   alternating rotation layers and entanglement layers. The rotation layers
   are single qubit gates applied on all qubits and the entanglement layer
   uses two-qubit gates to entangle the qubits according to a strategy set
   using ``entanglement``. Both the rotation and entanglement gates can be
   specified as string, gate-type or QuantumCircuit.
"""

"""This is a docstring for the imported function VQE
   Information below is based on the docstring in vqe.py

   Description
   -----------

   The Variational Quantum Eigensolver (VQE) algorithm, which is a quantum
   algorithm that uses a variational technique to find the minimum eigenvalue
   of the Hamiltonian :math:`H` of a given system. See more about
   VQE at https://arxiv.org/abs/1304.3061.

"""
another_solver = VQE(ansatz=tl_circuit,
                     quantum_instance=QuantumInstance(Aer.get_backend('aer_simulator_statevector')))

32:80: E501 line too long (100 > 79 characters)


### IV. The calculation and results

We are now ready to run the calculation as following:

In [10]:
from qiskit_nature.algorithms import GroundStateEigensolver
calc = GroundStateEigensolver(qubit_converter, vqe_solver)
"""This is a docstring for the imported function GroundStateEigensolver
   Information below is based on the docstring in ground_state_eigensolver.py

   Description
   -----------

   Ground state computation using a minimum eigensolver.
"""

"""This is a docstring for the function ElectronicStructureResult
   Information below is based on the docstring
   in electronic_structure_result.py

   Description
   -----------

   The electronic structure result.
"""
res = calc.solve(es_problem)

print(res)

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -1.857275030144
  - computed part:      -1.857275030144
~ Nuclear repulsion energy (Hartree): 0.719968994449
> Total ground state energy (Hartree): -1.137306035695
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 2.000 S: 0.000 S^2: -0.000 M: 0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.0  0.0  1.3889487]
 
  0: 
  * Electronic dipole moment (a.u.): [0.0  0.0  1.38894872]
    - computed part:      [0.0  0.0  1.38894872]
  > Dipole moment (a.u.): [0.0  0.0  -0.00000002]  Total: 0.00000002
                 (debye): [0.0  0.0  -0.00000004]  Total: 0.00000004
 


/opt/conda/lib/python3.8/site-packages/sympy/core/expr.py:3949: SymPyDeprecationWarning: 

expr_free_symbols method has been deprecated since SymPy 1.9. See
https://github.com/sympy/sympy/issues/21494 for more info.

  SymPyDeprecationWarning(feature="expr_free_symbols method",


We can compare the VQE results to the NumPy exact solver and see that they match.

In [11]:
calc = GroundStateEigensolver(qubit_converter, numpy_solver)
res = calc.solve(es_problem)
# The docstrings for these function QubitConverter are the same as above
print(res)

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -1.857275030202
  - computed part:      -1.857275030202
~ Nuclear repulsion energy (Hartree): 0.719968994449
> Total ground state energy (Hartree): -1.137306035753
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 2.000 S: 0.000 S^2: 0.000 M: 0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.0  0.0  1.3889487]
 
  0: 
  * Electronic dipole moment (a.u.): [0.0  0.0  1.3889487]
    - computed part:      [0.0  0.0  1.3889487]
  > Dipole moment (a.u.): [0.0  0.0  0.0]  Total: 0.0
                 (debye): [0.0  0.0  0.0]  Total: 0.0
 


In [12]:
import qiskit.tools.jupyter
%qiskit_version_table
%qiskit_copyright

/opt/conda/lib/python3.8/site-packages/qiskit/aqua/__init__.py:86: DeprecationWarning: The package qiskit.aqua is deprecated. It was moved/refactored to qiskit-terra For more information see <https://github.com/Qiskit/qiskit-aqua/blob/main/README.md#migration-guide>
  warn_package('aqua', 'qiskit-terra')


Qiskit Software,Version
qiskit-terra,0.18.3
qiskit-aer,0.9.1
qiskit-ignis,0.6.0
qiskit-ibmq-provider,0.17.0
qiskit-aqua,0.9.5
qiskit,0.31.0
qiskit-nature,0.2.2
qiskit-finance,0.2.1
qiskit-optimization,0.2.3
qiskit-machine-learning,0.2.1


**Note that the implementation part of this project is mainly based on tutorials from [Qiskit Nature Documentation](https://qiskit.org/documentation/nature/index.html)**. The References are listed below:

[[1] https://qiskit.org/documentation/nature/tutorials/01_electronic_structure.html](https://qiskit.org/documentation/nature/tutorials/01_electronic_structure.html);

[[2] https://qiskit.org/documentation/nature/tutorials/03_ground_state_solvers.html](https://qiskit.org/documentation/nature/tutorials/03_ground_state_solvers.html);